### Conversation Q&A Chatbot
In many Q&A applications we want to allow the user to have a back-and-forth conversation, meaning the application needs some sort of "memory" of past questions and answers, and some logic for incorporating those into its current thinking.

In this guide we focus on adding logic for incorporating historical messages. Further details on chat history management is covered in the previous videos.

We will cover two approaches:

- Chains, in which we always execute a retrieval step;
- Agents, in which we give an LLM discretion over whether and how to execute a retrieval step (or multiple steps).

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B628C27E50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B628C62290>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [4]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
import bs4
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_path="https://en.wikipedia.org/wiki/India_national_cricket_team",
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(id="mw-content-text")  # This holds the article body
    ),
)
docs = loader.load()


In [18]:
docs=loader.load()

In [19]:
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/India_national_cricket_team'}, page_content='Indian men\'s cricket team\nThis article is about the men\'s team. For the women\'s team, see India women\'s national cricket team.\n\n\nIndiaNickname(s)Men in Blue[1]AssociationBoard of Control for Cricket in IndiaPersonnelTest captainShubman GillOne Day captainRohit SharmaT20I captainSuryakumar YadavCoachGautam GambhirHistoryTest status acquired1931 (94\xa0years ago)\xa0(1931)International Cricket CouncilICC statusFull Member (1926)ICC regionACCICC\xa0Rankings\nCurrent[5]\nBest-everTest\n4th\n 1st(1973)[2]ODI\n 1st\n 1st(1994)[3]T20I\n 1st\n 1st(2014)[4]TestsFirst Testv \xa0England at Lord\'s, London; 25–28 June 1932Last Testv \xa0Australia at Sydney Cricket Ground, Sydney; 3–5 January 2025Tests\nPlayed\nWon/LostTotal[6]\n589\n181/184 (223 draws, 1 tie)This year[7]\n1\n0/1 (0 draws)World Test Championship appearances3 (first in 2021)Best result Runners-up(2021, 2023)One Day Inter

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
Vector_store=Chroma.from_documents(documents=splits,embedding=embeddings)
retriver = Vector_store.as_retriever()

In [21]:
## Prompt template
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [22]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

In [23]:
rag_chain = create_retrieval_chain(retriver,question_answer_chain)

In [32]:
response=rag_chain.invoke({"input": "what are some of the achievements of the India national cricket team?", "context": "India national cricket team"})
response

{'input': 'what are some of the achievements of the India national cricket team?',
 'context': [Document(id='9fe76074-9fa0-45c7-b20e-5b1eaad11dbb', metadata={'source': 'https://en.wikipedia.org/wiki/India_national_cricket_team'}, page_content="Indian men's cricket team\nThis article is about the men's team. For the women's team, see India women's national cricket team."),
  Document(id='82f7476b-d79c-420f-9123-efb9685201d2', metadata={'source': 'https://en.wikipedia.org/wiki/India_national_cricket_team'}, page_content='^ "Top 10 cricket matches at Eden Gardens, Kolkata". 21 September 2016. Archived from the original on 5 October 2022. Retrieved 5 October 2022.\n\n^ "Grounds: Feroz Shah Kotla". ESPNcricinfo. Archived from the original on 14 October 2006. Retrieved 8 March 2022.\n\n^ "Inaugural home venues of the 10 Test playing nations". 15 February 2017. Archived from the original on 15 April 2023. Retrieved 15 April 2023.\n\n^ "What Makes Mumbai Cricketers Special". Archived from the 

In [28]:
response['answer']

"I don't have personal opinions, but based on the context, MS Dhoni is ranked #1 in terms of most matches as captain in all formats with 332 matches."

## Adding history

In [40]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

# New prompt to use retrieved context to answer the question
answer_prompt = PromptTemplate(
    input_variables=["context", "input"],
    template="""
Use the following context to answer the question.

Context:
{context}

Question:
{input}

Answer:"""
)

# Now create the document-stuffing QA chain correctly
question_answer_chain = create_stuff_documents_chain(llm, answer_prompt)

In [41]:
history_aware_retriever = create_history_aware_retriever(
    retriever=retriver,
    llm=llm,
    prompt=contextualize_q_prompt
)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [50]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="Who was the captain when India won 2011 worldcup?"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="when did he retired?"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

According to the context, Mahendra Singh Dhoni was the captain when India won the 2011 Cricket World Cup.


In [51]:
chat_history

[HumanMessage(content='Who was the captain when India won 2011 worldcup?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the text, the captain when India won the 2011 Cricket World Cup was MS Dhoni.', additional_kwargs={}, response_metadata={})]